In [ ]:
!pip install pymorphy3
!pip install transformers
!pip install fse

In [63]:
import os # работа с папкой и файлами
import re # регулярные выражения, доп вариант к очистке
import string # работа со строкой
import math
import operator
import random
import time
import json
import datetime
import csv
import requests
import joblib

import pandas as pd
import numpy as np

from wordcloud import WordCloud, ImageColorGenerator # визуальное отображение
import pymorphy3 # работа с русским языком, pymorphy3

import matplotlib.pyplot as plt # визуальное отображение
import seaborn as sns
import plotly.express as px

from tqdm.auto import tqdm  # индикатор выполнения

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split # Деление выборки на тестовые и тренировочные данные
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

# python -m pip install transformers
from transformers import BertTokenizer # токенизатор BERT
from transformers import BertModel
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import DistilBertTokenizerFast
from transformers import BertForQuestionAnswering
from transformers import DistilBertForQuestionAnswering

import nltk # работа с пакетами языков
from nltk import word_tokenize, ngrams # токенизация и деление на n граммы
from nltk.corpus import stopwords # стопслова, extend

# python -m pip install tensorflow
# python -m pip install torch torchvision torchaudio
# import tensorflow as tf
# from tensorflow import keras
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score, roc_curve, auc, precision_score, recall_score, f1_score # Критерий качества, точности

import gensim
import gensim.downloader as api

# Для установки "pip install fse" необходимо:
# Python 3.10.13
# Visual Studio Build Tools 2022 (Включено: создание современных приложений C++ для Windows с помощью средств, включая MSVC, Clang, CMake или MSBuild)
from fse import SplitIndexedList
from fse.models import uSIF

# скачаем словарь стоп-слов
nltk.download("stopwords")

# скачиваем модель, которая будет делить на предложения
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sorokin.m.e\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sorokin.m.e\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Парсер файлов .json в DataFrame
def convert(f, path=['data', 'paragraphs', 'qas', 'answers']):
  file = json.loads(open(f).read())
  # разбор различных уровней в json-файле
  js = pd.json_normalize(file , path)
  m = pd.json_normalize(file, path[:-1])
  r = pd.json_normalize(file, path[:-2])
  # объединение их в один датафрейм
  m['context'] = np.repeat(r['context'].values, r.qas.str.len())
  js['q_idx'] = np.repeat(m['id'].values,m['answers'].str.len())
  data = m[['id', 'question', 'context', 'answers']].set_index('id').reset_index()
  # data = pd.concat([m.reset_index()[['id', 'question', 'context']].set_index('id'), js.reset_index().set_index('q_idx')], axis=1, sort=False).reset_index()
  data['context_id'] = data['context'].factorize()[0]
  return data

# Просмотр файлов в директории
print(os.listdir("archive"))

['dev-v1.1.json', 'dev-v2.0.json', 'train-v1.1.json', 'train-v2.0.json']


In [4]:
# Тренировочные данные
train = convert('archive/train-v1.1.json')
train

,id,question,context,answers,context_id
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...","[{'answer_start': 515, 'text': 'Saint Bernadet...",0
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...","[{'answer_start': 188, 'text': 'a copper statu...",0
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...","[{'answer_start': 279, 'text': 'the Main Build...",0
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...","[{'answer_start': 381, 'text': 'a Marian place...",0
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...","[{'answer_start': 92, 'text': 'a golden statue...",0
...,...,...,...,...,...
87594,5735d259012e2f140011a09d,In what US state did Kathmandu first establish...,"Kathmandu Metropolitan City (KMC), in order to...","[{'answer_start': 229, 'text': 'Oregon'}]",18890
87595,5735d259012e2f140011a09e,What was Yangon previously known as?,"Kathmandu Metropolitan City (KMC), in order to...","[{'answer_start': 414, 'text': 'Rangoon'}]",18890
87596,5735d259012e2f140011a09f,With what Belorussian city does Kathmandu have...,"Kathmandu Metropolitan City (KMC), in order to...","[{'answer_start': 476, 'text': 'Minsk'}]",18890
87597,5735d259012e2f140011a0a0,In what year did Kathmandu create its initial ...,"Kathmandu Metropolitan City (KMC), in order to...","[{'answer_start': 199, 'text': '1975'}]",18890


In [5]:
# Валидационные данные
valid = convert('archive/dev-v1.1.json')
valid

,id,question,context,answers,context_id
0,56be4db0acb8001400a502ec,Which NFL team represented the AFC at Super Bo...,Super Bowl 50 was an American football game to...,"[{'answer_start': 177, 'text': 'Denver Broncos...",0
1,56be4db0acb8001400a502ed,Which NFL team represented the NFC at Super Bo...,Super Bowl 50 was an American football game to...,"[{'answer_start': 249, 'text': 'Carolina Panth...",0
2,56be4db0acb8001400a502ee,Where did Super Bowl 50 take place?,Super Bowl 50 was an American football game to...,"[{'answer_start': 403, 'text': 'Santa Clara, C...",0
3,56be4db0acb8001400a502ef,Which NFL team won Super Bowl 50?,Super Bowl 50 was an American football game to...,"[{'answer_start': 177, 'text': 'Denver Broncos...",0
4,56be4db0acb8001400a502f0,What color was used to emphasize the 50th anni...,Super Bowl 50 was an American football game to...,"[{'answer_start': 488, 'text': 'gold'}, {'answ...",0
...,...,...,...,...,...
10565,5737aafd1c456719005744fb,What is the metric term less used than the New...,"The pound-force has a metric counterpart, less...","[{'answer_start': 82, 'text': 'kilogram-force'...",2066
10566,5737aafd1c456719005744fc,What is the kilogram-force sometimes reffered ...,"The pound-force has a metric counterpart, less...","[{'answer_start': 114, 'text': 'kilopond'}, {'...",2066
10567,5737aafd1c456719005744fd,What is a very seldom used unit of mass in the...,"The pound-force has a metric counterpart, less...","[{'answer_start': 274, 'text': 'slug'}, {'answ...",2066
10568,5737aafd1c456719005744fe,What seldom used term of a unit of force equal...,"The pound-force has a metric counterpart, less...","[{'answer_start': 712, 'text': 'kip'}, {'answe...",2066


In [6]:
# Проверка на заполненность вопросов тренировочных данных
for i, value in enumerate(train['answers']):
  data = json.loads(json.dumps(value[0]))
  if 'answer_start' not in data:
    print(value)
  if 'text' not in data:
    print(value)

In [7]:
# Проверка на заполненность вопросов валидационных данных
for i, value in enumerate(valid['answers']):
  data = json.loads(json.dumps(value[0]))
  if 'answer_start' not in data:
    print(value)
  if 'text' not in data:
    print(value)

In [8]:
# Размер тренировочных данных
print(f'Размер тренировочных данных: {train.shape}')

# Количество уникальных контекстов в наборе тренировочных данных
print(f'Количество уникальных контекстов в наборе тренировочных данных: {train["context_id"].unique().size}')

# Список уникальных контекстов в наборе тренировочных данных
train_unique = train[['context', 'context_id']].drop_duplicates().reset_index(drop=True)
train_unique

Размер тренировочных данных: (87599, 5)
Количество уникальных контекстов в наборе тренировочных данных: 18891


,context,context_id
0,"Architecturally, the school has a Catholic cha...",0
1,"As at most other universities, Notre Dame's st...",1
2,The university is the major seat of the Congre...,2
3,The College of Engineering was established in ...,3
4,All of Notre Dame's undergraduate students are...,4
...,...,...
18886,"Institute of Medicine, the central college of ...",18886
18887,Football and Cricket are the most popular spor...,18887
18888,The total length of roads in Nepal is recorded...,18888
18889,The main international airport serving Kathman...,18889


In [9]:
# Размер валидационных данных
print(f'Размер валидационных данных: {valid.shape}')

# Количество уникальных контекстов в наборе валидационных данных
print(f'Количество уникальных контекстов в наборе валидационных данных: {valid["context_id"].unique().size}')

# Список уникальных контекстов в наборе валидационных данных
valid_unique = valid[['context', 'context_id']].drop_duplicates().reset_index(drop=True)
valid_unique

Размер валидационных данных: (10570, 5)
Количество уникальных контекстов в наборе валидационных данных: 2067


,context,context_id
0,Super Bowl 50 was an American football game to...,0
1,The Panthers finished the regular season with ...,1
2,The Broncos took an early lead in Super Bowl 5...,2
3,"CBS broadcast Super Bowl 50 in the U.S., and c...",3
4,"In early 2012, NFL Commissioner Roger Goodell ...",4
...,...,...
2062,"where is the mass of the object, is the velo...",2062
2063,A conservative force that acts on a closed sys...,2063
2064,"For certain physical scenarios, it is impossib...",2064
2065,The connection between macroscopic nonconserva...,2065


In [10]:
# Контексты тренировочных данных
train_contexts = train['context'].to_numpy().tolist()
train_contexts

['Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building

In [11]:
# Контексты валидационных данных
valid_contexts = valid['context'].to_numpy().tolist()
valid_contexts

['Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Foo

In [12]:
# Вопросы тренировочных данных
train_questions = train['question'].to_numpy().tolist()
train_questions

['To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'What is in front of the Notre Dame Main Building?',
 'The Basilica of the Sacred heart at Notre Dame is beside to which structure?',
 'What is the Grotto at Notre Dame?',
 'What sits on top of the Main Building at Notre Dame?',
 'When did the Scholastic Magazine of Notre dame begin publishing?',
 "How often is Notre Dame's the Juggler published?",
 'What is the daily student paper at Notre Dame called?',
 'How many student news papers are found at Notre Dame?',
 'In what year did the student paper Common Sense begin publication at Notre Dame?',
 'Where is the headquarters of the Congregation of the Holy Cross?',
 'What is the primary seminary of the Congregation of the Holy Cross?',
 'What is the oldest structure at Notre Dame?',
 'What individuals live at Fatima House at Notre Dame?',
 'Which prize did Frederick Buechner create?',
 'How many BS level degrees are offered in the College of Engineering at Notre

In [13]:
# Вопросы валидационных данных
valid_questions = valid['question'].to_numpy().tolist()
valid_questions

['Which NFL team represented the AFC at Super Bowl 50?',
 'Which NFL team represented the NFC at Super Bowl 50?',
 'Where did Super Bowl 50 take place?',
 'Which NFL team won Super Bowl 50?',
 'What color was used to emphasize the 50th anniversary of the Super Bowl?',
 'What was the theme of Super Bowl 50?',
 'What day was the game played on?',
 'What is the AFC short for?',
 'What was the theme of Super Bowl 50?',
 'What does AFC stand for?',
 'What day was the Super Bowl played on?',
 'Who won Super Bowl 50?',
 'What venue did Super Bowl 50 take place in?',
 'What city did Super Bowl 50 take place in?',
 'If Roman numerals were used, what would Super Bowl 50 have been called?',
 'Super Bowl 50 decided the NFL champion for what season?',
 'What year did the Denver Broncos secure a Super Bowl title for the third time?',
 'What city did Super Bowl 50 take place in?',
 'What stadium did Super Bowl 50 take place in?',
 'What was the final score of Super Bowl 50? ',
 'What month, day and y

In [14]:
# Ответы тренировочных данных
train_answers = []
for i, value in enumerate(train['answers'].to_numpy().tolist()):
  train_answers.append(value[0]['text'])
train_answers

['Saint Bernadette Soubirous',
 'a copper statue of Christ',
 'the Main Building',
 'a Marian place of prayer and reflection',
 'a golden statue of the Virgin Mary',
 'September 1876',
 'twice',
 'The Observer',
 'three',
 '1987',
 'Rome',
 'Moreau Seminary',
 'Old College',
 'Retired priests and brothers',
 'Buechner Prize for Preaching',
 'eight',
 '1920',
 'the College of Science',
 'five',
 'the 1870s',
 'Learning Resource Center',
 'five',
 'The First Year of Studies program',
 'U.S. News & World Report',
 '1924',
 'Master of Divinity',
 'Alliance for Catholic Education',
 '1854',
 'Department of Pre-Professional Studies',
 'Joan B. Kroc Institute for International Peace Studies',
 'President Emeritus of the University of Notre Dame',
 '1986',
 'Ray Kroc',
 "McDonald's",
 '14',
 'Theodore M. Hesburgh Library',
 '1963',
 'Millard Sheets',
 'Touchdown Jesus',
 '3,577',
 '19.7%',
 'the top 10 to 15 in the nation',
 '39.1%',
 'more than 750 miles',
 '18th overall',
 '8th',
 '1st overa

In [15]:
# Ответы валидационных данных
valid_answers = []
for i, value in enumerate(valid['answers'].to_numpy().tolist()):
  valid_answers.append(value[0]['text'])
valid_answers

['Denver Broncos',
 'Carolina Panthers',
 'Santa Clara, California',
 'Denver Broncos',
 'gold',
 '"golden anniversary"',
 'February 7, 2016',
 'American Football Conference',
 '"golden anniversary"',
 'American Football Conference',
 'February 7, 2016',
 'Denver Broncos',
 "Levi's Stadium",
 'Santa Clara',
 'Super Bowl L',
 '2015',
 '2015',
 'Santa Clara',
 "Levi's Stadium",
 '24–10',
 'February 7, 2016',
 '2015',
 'Denver Broncos',
 'Carolina Panthers',
 'Denver Broncos',
 '2015',
 'Denver Broncos',
 'Santa Clara, California.',
 'Super Bowl',
 'Denver Broncos',
 'Cam Newton',
 '8',
 '1995',
 'Arizona Cardinals',
 'New England Patriots',
 'Arizona Cardinals',
 'New England Patriots',
 'New England Patriots',
 'four',
 'Cam Newton',
 '15–1',
 'Cam Newton',
 '12–4',
 '4',
 'New England Patriots',
 'Cam Newton',
 'Arizona Cardinals',
 '2',
 'New England Patriots',
 'Cam Newton',
 'New England Patriots',
 'Arizona Cardinals',
 'Cam Newton',
 'Arizona Cardinals',
 '1995.',
 'Von Miller',
 

In [16]:
# Общие тренировочные данные
train_merge = pd.merge(pd.DataFrame(train_contexts, columns=['contexts']), pd.DataFrame(train_questions, columns=['questions']), left_index=True, right_index=True)
train_data = pd.merge(train_merge, pd.DataFrame(train_answers, columns=['answers']), left_index=True, right_index=True)
train_data

,contexts,questions,answers
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary
...,...,...,...
87594,"Kathmandu Metropolitan City (KMC), in order to...",In what US state did Kathmandu first establish...,Oregon
87595,"Kathmandu Metropolitan City (KMC), in order to...",What was Yangon previously known as?,Rangoon
87596,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,Minsk
87597,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,1975


In [17]:
# Общие валидационные данные
valid_merge = pd.merge(pd.DataFrame(valid_contexts, columns=['contexts']), pd.DataFrame(valid_questions, columns=['questions']), left_index=True, right_index=True)
valid_data = pd.merge(valid_merge, pd.DataFrame(valid_answers, columns=['answers']), left_index=True, right_index=True)
valid_data

,contexts,questions,answers
0,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,Denver Broncos
1,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...,Carolina Panthers
2,Super Bowl 50 was an American football game to...,Where did Super Bowl 50 take place?,"Santa Clara, California"
3,Super Bowl 50 was an American football game to...,Which NFL team won Super Bowl 50?,Denver Broncos
4,Super Bowl 50 was an American football game to...,What color was used to emphasize the 50th anni...,gold
...,...,...,...
10565,"The pound-force has a metric counterpart, less...",What is the metric term less used than the New...,kilogram-force
10566,"The pound-force has a metric counterpart, less...",What is the kilogram-force sometimes reffered ...,kilopond
10567,"The pound-force has a metric counterpart, less...",What is a very seldom used unit of mass in the...,slug
10568,"The pound-force has a metric counterpart, less...",What seldom used term of a unit of force equal...,kip


In [18]:
# Исправление символов конечного положения в тренировочных данных
for answer, text in zip(train['answers'].values.tolist(), train['context'].values.tolist()):
  real_answer = answer[0]['text']
  answer_start_index = answer[0]['answer_start']
  answer_end_index = answer_start_index + len(real_answer)
  if text[answer_start_index : answer_end_index] == real_answer:
    answer[0]['answer_end'] = answer_end_index
  elif text[answer_start_index - 1 : answer_end_index - 1] == real_answer:
    answer[0]['answer_start'] = answer_start_index - 1
    answer[0]['answer_end'] = answer_end_index - 1
  elif text[answer_start_index - 2 : answer_end_index - 2] == real_answer:
    answer[0]['answer_start'] = answer_start_index - 2
    answer[0]['answer_end'] = answer_end_index - 2

train_answers_positions = []
for i, value in enumerate(train['answers'].values.tolist()):
  temp = []
  for j, item in enumerate(value):
    if 'answer_end' in item:
      temp.append(item['answer_start'])
      temp.append(item['text'])
      temp.append(item['answer_end'])
  train_answers_positions.append(temp)

pd.DataFrame(train_answers_positions, columns=['answer_start', 'text', 'answer_end'])

,answer_start,text,answer_end
0,515,Saint Bernadette Soubirous,541
1,188,a copper statue of Christ,213
2,279,the Main Building,296
3,381,a Marian place of prayer and reflection,420
4,92,a golden statue of the Virgin Mary,126
...,...,...,...
87594,229,Oregon,235
87595,414,Rangoon,421
87596,476,Minsk,481
87597,199,1975,203


In [19]:
# Исправление символов конечного положения в валидационных данных
for answer, text in zip(valid['answers'].values.tolist(), valid['context'].values.tolist()):
  real_answer = answer[0]['text']
  answer_start_index = answer[0]['answer_start']
  answer_end_index = answer_start_index + len(real_answer)
  if text[answer_start_index : answer_end_index] == real_answer:
    answer[0]['answer_end'] = answer_end_index
  elif text[answer_start_index - 1 : answer_end_index - 1] == real_answer:
    answer[0]['answer_start'] = answer_start_index - 1
    answer[0]['answer_end'] = answer_end_index - 1
  elif text[answer_start_index - 2 : answer_end_index - 2] == real_answer:
    answer[0]['answer_start'] = answer_start_index - 2
    answer[0]['answer_end'] = answer_end_index - 2

valid_answers_positions = []
for i, value in enumerate(valid['answers'].values.tolist()):
  temp = []
  for j, item in enumerate(value):
    if 'answer_end' in item:
      temp.append(item['answer_start'])
      temp.append(item['text'])
      temp.append(item['answer_end'])
  valid_answers_positions.append(temp)

pd.DataFrame(valid_answers_positions, columns=['answer_start', 'text', 'answer_end'])

,answer_start,text,answer_end
0,177,Denver Broncos,191
1,249,Carolina Panthers,266
2,403,"Santa Clara, California",426
3,177,Denver Broncos,191
4,488,gold,492
...,...,...,...
10565,82,kilogram-force,96
10566,114,kilopond,122
10567,274,slug,278
10568,712,kip,715


In [26]:
# Общие тренировочные данные с ответами
train_data_answers = pd.merge(train, pd.DataFrame(train_answers_positions, columns=['answer_start', 'text', 'answer_end']), left_index=True, right_index=True)
train_data_answers

,id,question,context,answers,context_id,answer_start,text,answer_end
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...","[{'answer_start': 515, 'text': 'Saint Bernadet...",0,515,Saint Bernadette Soubirous,541
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...","[{'answer_start': 188, 'text': 'a copper statu...",0,188,a copper statue of Christ,213
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...","[{'answer_start': 279, 'text': 'the Main Build...",0,279,the Main Building,296
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...","[{'answer_start': 381, 'text': 'a Marian place...",0,381,a Marian place of prayer and reflection,420
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...","[{'answer_start': 92, 'text': 'a golden statue...",0,92,a golden statue of the Virgin Mary,126
...,...,...,...,...,...,...,...,...
87594,5735d259012e2f140011a09d,In what US state did Kathmandu first establish...,"Kathmandu Metropolitan City (KMC), in order to...","[{'answer_start': 229, 'text': 'Oregon', 'answ...",18890,229,Oregon,235
87595,5735d259012e2f140011a09e,What was Yangon previously known as?,"Kathmandu Metropolitan City (KMC), in order to...","[{'answer_start': 414, 'text': 'Rangoon', 'ans...",18890,414,Rangoon,421
87596,5735d259012e2f140011a09f,With what Belorussian city does Kathmandu have...,"Kathmandu Metropolitan City (KMC), in order to...","[{'answer_start': 476, 'text': 'Minsk', 'answe...",18890,476,Minsk,481
87597,5735d259012e2f140011a0a0,In what year did Kathmandu create its initial ...,"Kathmandu Metropolitan City (KMC), in order to...","[{'answer_start': 199, 'text': '1975', 'answer...",18890,199,1975,203


In [21]:
# Общие валидационные данные с ответами
valid_data_answers = pd.merge(valid, pd.DataFrame(valid_answers_positions, columns=['answer_start', 'text', 'answer_end']), left_index=True, right_index=True)
valid_data_answers

,id,question,context,answers,context_id,answer_start,text,answer_end
0,56be4db0acb8001400a502ec,Which NFL team represented the AFC at Super Bo...,Super Bowl 50 was an American football game to...,"[{'answer_start': 177, 'text': 'Denver Broncos...",0,177,Denver Broncos,191
1,56be4db0acb8001400a502ed,Which NFL team represented the NFC at Super Bo...,Super Bowl 50 was an American football game to...,"[{'answer_start': 249, 'text': 'Carolina Panth...",0,249,Carolina Panthers,266
2,56be4db0acb8001400a502ee,Where did Super Bowl 50 take place?,Super Bowl 50 was an American football game to...,"[{'answer_start': 403, 'text': 'Santa Clara, C...",0,403,"Santa Clara, California",426
3,56be4db0acb8001400a502ef,Which NFL team won Super Bowl 50?,Super Bowl 50 was an American football game to...,"[{'answer_start': 177, 'text': 'Denver Broncos...",0,177,Denver Broncos,191
4,56be4db0acb8001400a502f0,What color was used to emphasize the 50th anni...,Super Bowl 50 was an American football game to...,"[{'answer_start': 488, 'text': 'gold', 'answer...",0,488,gold,492
...,...,...,...,...,...,...,...,...
10565,5737aafd1c456719005744fb,What is the metric term less used than the New...,"The pound-force has a metric counterpart, less...","[{'answer_start': 82, 'text': 'kilogram-force'...",2066,82,kilogram-force,96
10566,5737aafd1c456719005744fc,What is the kilogram-force sometimes reffered ...,"The pound-force has a metric counterpart, less...","[{'answer_start': 114, 'text': 'kilopond', 'an...",2066,114,kilopond,122
10567,5737aafd1c456719005744fd,What is a very seldom used unit of mass in the...,"The pound-force has a metric counterpart, less...","[{'answer_start': 274, 'text': 'slug', 'answer...",2066,274,slug,278
10568,5737aafd1c456719005744fe,What seldom used term of a unit of force equal...,"The pound-force has a metric counterpart, less...","[{'answer_start': 712, 'text': 'kip', 'answer_...",2066,712,kip,715


In [27]:
# Очистка тескта
def clean_text(text):
  text = str(text)
  text = text.lower()
  words = re.sub(r'[^\w\s\.\?]', '', text).split()
  return " ".join([word for word in words])

train_data_answers['question'] = train_data_answers['question'].apply(lambda x: clean_text(x))
train_data_answers['context'] = train_data_answers['context'].apply(lambda x: clean_text(x))
train_data_answers['text'] = train_data_answers['text'].apply(lambda x: clean_text(x))

train_all_question = '. '.join(list(train_data_answers['question']))
train_all_answer = '. '.join(list(train_data_answers['text']))
train_all_context = '. '.join(list(train_data_answers['context']))
train_text = train_all_question + train_all_answer + train_all_context
train_split = SplitIndexedList(train_text.split('.'))

valid_data_answers['question'] = valid_data_answers['question'].apply(lambda x: clean_text(x))
valid_data_answers['context'] = valid_data_answers['context'].apply(lambda x: clean_text(x))
valid_data_answers['text'] = valid_data_answers['text'].apply(lambda x: clean_text(x))

valid_all_question = '. '.join(list(valid_data_answers['question']))
valid_all_answer = '. '.join(list(valid_data_answers['text']))
valid_all_context = '. '.join(list(valid_data_answers['context']))
valid_text = valid_all_question + valid_all_answer + valid_all_context
valid_split = SplitIndexedList(valid_text.split('.'))

In [28]:
# Разделение на слова, преобразование для обучения тренировочных данных
train_dataset = []
train_title = ""

for i in range(0, len(train_data_answers), 2):
  this_title = train_data_answers.iloc[i]['id']
  if (this_title != train_title):
    train_title = this_title
    text = train_data_answers.iloc[i]['context']
    splitted = text.split(sep='.')
    for j in range(len(splitted)):
      text = splitted[j]
      if(text!=''):
        words = text.split()
        train_dataset.append(words)
  train_dataset.append(train_data_answers.iloc[i]['question'].split())
  train_dataset.append(train_data_answers.iloc[i]['text'].split())
train_dataset

[['architecturally', 'the', 'school', 'has', 'a', 'catholic', 'character'],
 ['atop',
  'the',
  'main',
  'buildings',
  'gold',
  'dome',
  'is',
  'a',
  'golden',
  'statue',
  'of',
  'the',
  'virgin',
  'mary'],
 ['immediately',
  'in',
  'front',
  'of',
  'the',
  'main',
  'building',
  'and',
  'facing',
  'it',
  'is',
  'a',
  'copper',
  'statue',
  'of',
  'christ',
  'with',
  'arms',
  'upraised',
  'with',
  'the',
  'legend',
  'venite',
  'ad',
  'me',
  'omnes'],
 ['next',
  'to',
  'the',
  'main',
  'building',
  'is',
  'the',
  'basilica',
  'of',
  'the',
  'sacred',
  'heart'],
 ['immediately',
  'behind',
  'the',
  'basilica',
  'is',
  'the',
  'grotto',
  'a',
  'marian',
  'place',
  'of',
  'prayer',
  'and',
  'reflection'],
 ['it',
  'is',
  'a',
  'replica',
  'of',
  'the',
  'grotto',
  'at',
  'lourdes',
  'france',
  'where',
  'the',
  'virgin',
  'mary',
  'reputedly',
  'appeared',
  'to',
  'saint',
  'bernadette',
  'soubirous',
  'in',
  '1

In [29]:
# Разделение на слова, преобразование для обучения валидационных данных
valid_dataset = []
valid_title = ""

for i in range(0, len(valid_data_answers), 2):
  this_title = valid_data_answers.iloc[i]['id']
  if (this_title != valid_title):
    valid_title = this_title
    text = valid_data_answers.iloc[i]['context']
    splitted = text.split(sep='.')
    for j in range(len(splitted)):
      text = splitted[j]
      if(text!=''):
        words = text.split()
        valid_dataset.append(words)
  valid_dataset.append(valid_data_answers.iloc[i]['question'].split())
  valid_dataset.append(valid_data_answers.iloc[i]['text'].split())
valid_dataset

[['super',
  'bowl',
  '50',
  'was',
  'an',
  'american',
  'football',
  'game',
  'to',
  'determine',
  'the',
  'champion',
  'of',
  'the',
  'national',
  'football',
  'league',
  'nfl',
  'for',
  'the',
  '2015',
  'season'],
 ['the',
  'american',
  'football',
  'conference',
  'afc',
  'champion',
  'denver',
  'broncos',
  'defeated',
  'the',
  'national',
  'football',
  'conference',
  'nfc',
  'champion',
  'carolina',
  'panthers',
  '2410',
  'to',
  'earn',
  'their',
  'third',
  'super',
  'bowl',
  'title'],
 ['the',
  'game',
  'was',
  'played',
  'on',
  'february',
  '7',
  '2016',
  'at',
  'levis',
  'stadium',
  'in',
  'the',
  'san',
  'francisco',
  'bay',
  'area',
  'at',
  'santa',
  'clara',
  'california'],
 ['as',
  'this',
  'was',
  'the',
  '50th',
  'super',
  'bowl',
  'the',
  'league',
  'emphasized',
  'the',
  'golden',
  'anniversary',
  'with',
  'various',
  'goldthemed',
  'initiatives',
  'as',
  'well',
  'as',
  'temporarily',
  

In [30]:
# Word2Vec и обучение модели для тренировочных данных
train_model_wv = gensim.models.Word2Vec(train_dataset, vector_size=100, window=8, min_count=1, sg=0, workers=8)
train_model_wv.train(train_dataset, total_examples=len(train_dataset), compute_loss=True, epochs=50)

(228825926, 291597050)

In [33]:
# Word2Vec и обучение модели для валидационных данных
valid_model_wv = gensim.models.Word2Vec(valid_dataset, vector_size=100, window=8, min_count=1, sg=0, workers=8)
valid_model_wv.train(valid_dataset, total_examples=len(valid_dataset), compute_loss=True, epochs=50)

(28563358, 36348850)

In [43]:
def get_embedding(model_wv, sentence):
  pos_sum = [0.0 for i in range(100)]
  num = 0
  words = sentence.split()
  for i in words:
    try:
      embed = model_wv.wv[i]
    except:
      continue
    else:
      pos_sum += embed
      num +=1
  if(num==0):
    return pos_sum
  else:
    pos_sum /= num
    return pos_sum

# Предсказание модели с использованием евклидова расстояния
def get_answer(model_wv, question, answer_para):
  question_embedding = get_embedding(model_wv, rem_stop(question))
  min_distance = math.inf
  answer = 0
  for i in range(len(answer_para)):
    answer_embedding = get_embedding(model_wv, rem_stop(answer_para[i]))
    distance = np.linalg.norm(question_embedding-answer_embedding)
    if (distance < min_distance):
      answer = i
      # print(answer)
      min_distance = distance
  return answer_para[answer]

# Без стоп слов
def rem_stop(sentence):
  strr=''
  my_string = sentence.split()
  for i in range(len(my_string)):
    if my_string[i] not in stopwords.words('english'):
      strr = strr+' '+my_string[i]
  return strr[1:]

# Предсказание модели с использованием косинусного сходства
def get_answer_cosine(model_wv, question, answer_para):
  question_embedding = get_embedding(model_wv, rem_stop(question))
  max_similarity = -math.inf
  answer = 0
  for i in range(len(answer_para)):
    answer_embedding = get_embedding(model_wv, rem_stop(answer_para[i]))
    similarity = cosine_similarity(np.expand_dims(question_embedding,0), np.expand_dims(answer_embedding,0))
    if (similarity > max_similarity):
      answer = i
      max_similarity = similarity
  return answer_para[answer]

# Разбиваем текст статьи на отдельные предложения
def contextToSents(my_text):
  temp_sentences = my_text.split(sep='.')
  sentences=[]
  for i in range(len(temp_sentences)):
    if(temp_sentences[i]!=''):
      sentences.append(temp_sentences[i])
  return sentences

In [37]:
# Вопрос из тренировочных данных
index = 296
my_text = train_data_answers.iloc[index]['context']
temp_sentences = my_text.split(sep='.')
train_sentences = []

for i in range(len(temp_sentences)):
  if(temp_sentences[i]!=''):
    train_sentences.append(temp_sentences[i])

train_my_question = train_data_answers.iloc[index]['question']
train_my_question

'when did destinys child end their group act?'

In [38]:
# Вопрос из валидационных данных
index = 296
my_text = valid_data_answers.iloc[index]['context']
temp_sentences = my_text.split(sep='.')
valid_sentences = []

for i in range(len(temp_sentences)):
  if(temp_sentences[i]!=''):
    valid_sentences.append(temp_sentences[i])

valid_my_question = valid_data_answers.iloc[index]['question']
valid_my_question

'how many points did the broncos defense allow their opponents to get?'

In [48]:
# Word2Vec outputs из тренировочных данных
print(f'Вопрос:\n{train_my_question}')
print(f'\nВопрос без стоп слов:\n{rem_stop(train_my_question)}')
print(f'\nОтвет:\n{train_data_answers.iloc[index]["text"]}')
print(f'\nПредсказание модели с использованием евклидова расстояния:\n{get_answer(train_model_wv, train_my_question, train_sentences)}')
print(f'\nПредсказание модели с использованием косинусного сходства:\n{get_answer_cosine(train_model_wv, train_my_question, train_sentences)}')

Вопрос:
when did destinys child end their group act?

Вопрос без стоп слов:
destinys child end group act?

Ответ:
june 2005

Предсказание модели с использованием евклидова расстояния:
following the disbandment of destinys child in june 2005 she released her second solo album bday 2006 which contained hits déjà vu irreplaceable and beautiful liar

Предсказание модели с использованием косинусного сходства:
following the disbandment of destinys child in june 2005 she released her second solo album bday 2006 which contained hits déjà vu irreplaceable and beautiful liar


In [49]:
# Word2Vec outputs из валидационных данных
print(f'Вопрос:\n{valid_my_question}')
print(f'\nВопрос без стоп слов:\n{rem_stop(valid_my_question)}')
print(f'\nОтвет:\n{valid_data_answers.iloc[index]["text"]}')
print(f'\nПредсказание модели с использованием евклидова расстояния:\n{get_answer(valid_model_wv, valid_my_question, valid_sentences)}')
print(f'\nПредсказание модели с использованием косинусного сходства:\n{get_answer_cosine(valid_model_wv, valid_my_question, valid_sentences)}')

Вопрос:
how many points did the broncos defense allow their opponents to get?

Вопрос без стоп слов:
many points broncos defense allow opponents get?

Ответ:
296

Предсказание модели с использованием евклидова расстояния:
the broncos defense ranked first in the nfl yards allowed 4530 for the first time in franchise history and fourth in points allowed 296

Предсказание модели с использованием косинусного сходства:
the broncos defense ranked first in the nfl yards allowed 4530 for the first time in franchise history and fourth in points allowed 296


In [52]:
# Загрузка предтренированной модели BERT
# model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [53]:
# Загрузка предтренированного токенайзера BERT
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [54]:
# Базовая конфигурация предтренированного токенайзера BERT
model.base_model.config

BertConfig {
  "_name_or_path": "bert-large-uncased-whole-word-masking-finetuned-squad",
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.35.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [55]:
pytorch_total_params = sum(p.numel() for p in model.base_model.parameters() if p.requires_grad)
pytorch_trainable_params = sum(p.numel() for p in model.base_model.parameters() )
print("Total number of params", pytorch_total_params)
print("Total number of trainable params", pytorch_trainable_params)

Total number of params 334092288
Total number of trainable params 334092288


In [56]:
# Токенайзер тренировочных данных
train_encodings = tokenizer(
  train_data['contexts'].values.tolist(),
  train_data['questions'].values.tolist(),
  truncation=True,
  padding=True,
  max_length=512,
  return_tensors='pt'
)
train_encodings

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

{'input_ids': tensor([[  101,  6549,  2135,  ...,     0,     0,     0],
        [  101,  6549,  2135,  ...,     0,     0,     0],
        [  101,  6549,  2135,  ...,     0,     0,     0],
        ...,
        [  101, 28045,  4956,  ...,     0,     0,     0],
        [  101, 28045,  4956,  ...,     0,     0,     0],
        [  101, 28045,  4956,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [57]:
# Токенайзер валидационных данных
validation_encodings = tokenizer(
  valid_data['contexts'].values.tolist(),
  valid_data['questions'].values.tolist(),
  truncation=True,
  padding=True,
  max_length=512,
  return_tensors='pt'
)
validation_encodings

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

{'input_ids': tensor([[ 101, 3565, 4605,  ...,    0,    0,    0],
        [ 101, 3565, 4605,  ...,    0,    0,    0],
        [ 101, 3565, 4605,  ...,    0,    0,    0],
        ...,
        [ 101, 1996, 9044,  ...,    0,    0,    0],
        [ 101, 1996, 9044,  ...,    0,    0,    0],
        [ 101, 1996, 9044,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [58]:
def get_top_answers(possible_starts, possible_ends, input_ids):
  answers = []
  for start,end in zip(possible_starts, possible_ends):
    #+1 for end
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[start:end+1]))
    answers.append( answer )
  return answers

def answer_question(question, context, topN):
  inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
  input_ids = inputs["input_ids"].tolist()[0]

  text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
  model_out = model(**inputs)

  answer_start_scores = model_out["start_logits"]
  answer_end_scores = model_out["end_logits"]

  possible_starts = np.argsort(answer_start_scores.cpu().detach().numpy()).flatten()[::-1][:topN]
  possible_ends = np.argsort(answer_end_scores.cpu().detach().numpy()).flatten()[::-1][:topN]

  #get best answer
  answer_start = torch.argmax(answer_start_scores)
  answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score
  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
  answers = get_top_answers(possible_starts, possible_ends, input_ids)

  return {
    "answer":answer,
    "answer_start":answer_start,
    "answer_end":answer_end,
    "input_ids":input_ids,
    "answer_start_scores":answer_start_scores,
    "answer_end_scores":answer_end_scores,
    "inputs":inputs,
    "answers":answers,
    "possible_starts":possible_starts,
    "possible_ends":possible_ends
  }

In [101]:
index = 0

# Контекст
text = train_data_answers.iloc[index]['context']
print(f'Контекст:\n{text}')

# Список вопросов
questions = []
for i, value in enumerate(train_data_answers['question']):
  if train_data_answers['context_id'][i] == train_data_answers.iloc[index]['context_id']:
    questions.append(value)
print(f'\nВопросы:\n{questions}')

# Список ответов
for q in questions:
  answer_map = answer_question(q, text, 5)
  print("\nВопрос:", q)
  print("Ответы:")
  [print((index+1), ") ", ans) for index, ans in enumerate(answer_map["answers"]) if len(ans) > 0]

Контекст:
architecturally the school has a catholic character. atop the main buildings gold dome is a golden statue of the virgin mary. immediately in front of the main building and facing it is a copper statue of christ with arms upraised with the legend venite ad me omnes. next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto a marian place of prayer and reflection. it is a replica of the grotto at lourdes france where the virgin mary reputedly appeared to saint bernadette soubirous in 1858. at the end of the main drive and in a direct line that connects through 3 statues and the gold dome is a simple modern stone statue of mary.

Вопросы:
['to whom did the virgin mary allegedly appear in 1858 in lourdes france?', 'what is in front of the notre dame main building?', 'the basilica of the sacred heart at notre dame is beside to which structure?', 'what is the grotto at notre dame?', 'what sits on top of the main building at notre d

In [65]:
# Выгружаем модель
torch.save(model, f"model.pth")

In [66]:
# Загружаем натренированную модель
bert = torch.load(f"model.pth")
bert.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), ep